In [13]:
import chromadb
import asyncio
import ollama

In [14]:
chroma_client = chromadb.HttpClient(host='localhost', port=8000)

# collection = chroma_client.get_or_create_collection(name='docs')

In [15]:
collection = chroma_client.get_or_create_collection(name='docs')

In [4]:
# 預先將會重複使用的功能，以函數的方式，便於後續呼叫使用

In [17]:
def heartbeat():
    return chroma_client.heartbeat()

def delete_collection(collection):
    chroma_client.delete_collection(collection)

def collection_reset():
    data_ids = collection.peek(100)['ids']
    for i in data_ids:
        # print(i)
        collection.delete(i)

def collection_add(i, embedding, document):
    collection.add(
        ids=[i],
        embeddings=[embedding],
        documents=[document]
    )

def collection_query(query, n_results):
    results = collection.query(
        query_embeddings=query,
        n_results=n_results
    )
    return results

In [24]:
while True:
    data_ids = collection.peek()['ids']
    if not data_ids:
        break
    for i in data_ids:
        # print(i)
        collection.delete(i)

In [22]:
documents = [
    "動物界,脊索動物門,哺乳綱,嚙齒目,鼠科,大鼠屬,溝鼠",
    "動物界,脊索動物門,哺乳綱,嚙齒目,倉鼠科,田鼠屬,田鼠",
    "動物界,脊索動物門,哺乳綱,嚙齒目,松鼠科,麗松鼠屬,赤腹鼠",
    "動物界,脊索動物門,哺乳綱,雙門齒目,袋鼠科,袋鼠屬,紅大袋鼠",
    "動物界,脊索動物門,哺乳綱,偶蹄目,牛科,牛屬,黃牛",
    "動物界,脊索動物門,哺乳綱,偶蹄目,牛科,水牛屬,水牛",
    "動物界,脊索動物門,哺乳綱,肉食目,貓科,豹屬,孟加拉虎",
    "動物界,脊索動物門,哺乳綱,肉食目,貓科,豹屬,獅子",
    "動物界,脊索動物門,哺乳綱,肉食目,貓科,豹屬,花豹",
    "動物界,脊索動物門,哺乳綱,肉食目,貓科,石虎屬,石虎",
    "動物界,脊索動物門,哺乳綱,肉食目,貓科,獵豹屬,獵豹",
    "動物界,脊索動物門,哺乳綱,兔形目,兔科,穴兔屬,家兔",
    "動物界,脊索動物門,爬蟲綱,有鱗目,眼鏡蛇科,環蛇屬,雨傘節",
    "動物界,脊索動物門,爬蟲綱,有鱗目,蝰科,青竹絲屬,青竹絲",
    "動物界,脊索動物門,哺乳綱,奇蹄目,馬科,馬屬,馬",
    "動物界,脊索動物門,哺乳綱,奇蹄目,馬科,馬屬,斑馬",
    "動物界,脊索動物門,哺乳綱,偶蹄目,牛科,山羊屬,山羊",
    "動物界,脊索動物門,哺乳綱,偶蹄目,牛科,盤羊屬,綿羊",
    "動物界,脊索動物門,哺乳綱,偶蹄目,牛科,鬣羚屬,台灣長鬃山羊",
    "動物界,脊索動物門,哺乳綱,靈長目,猴科,獼猴屬,台灣獼猴",
]
# 檢查每個文件 ID 是否已存在，如果存在則跳過新增操作
existing_docs = collection.get()
existing_ids = set(existing_docs['ids'])

for i, d in enumerate(documents):
    response = ollama.embeddings(model="llama3", prompt=d)
    embedding = response["embedding"]
    collection_add(str(i),embedding,d)
    # print(f"embeddings: [{embedding}], documents: [{d}]")
    # collection.add(
    #   ids=[str(i)],
    #   embeddings=[embedding],
    #   documents=[d]
    # )

print('done')


done


In [93]:
prompt = "資料庫中哪中動物與下列這個最相近\n'動物界,脊索動物門,哺乳綱,偶蹄目,牛科,狗狗屬,白狗'?"
response = ollama.embeddings(
  prompt=prompt,
  model="llama3"
)

In [19]:
results = collection_query([response["embedding"]],6)
data = results['documents'][0]
print(f"data: {data}")
print(results)

data: ['動物界,脊索動物門,哺乳綱,靈長目,猴科,獼猴屬,台灣獼猴', '動物界,脊索動物門,哺乳綱,雙門齒目,袋鼠科,袋鼠屬,紅大袋鼠', '動物界,脊索動物門,哺乳綱,偶蹄目,牛科,鬣羚屬,台灣長鬃山羊', '動物界,脊索動物門,哺乳綱,肉食目,貓科,豹屬,花豹', '動物界,脊索動物門,哺乳綱,肉食目,貓科,石虎屬,石虎', '動物界,脊索動物門,哺乳綱,兔形目,兔科,穴兔屬,家兔']
{'ids': [['19', '3', '18', '8', '9', '11']], 'distances': [[0.0, 3424.0587441161697, 4206.558097385314, 4765.787329230293, 5247.593207506797, 5561.906275189448]], 'embeddings': None, 'metadatas': [[None, None, None, None, None, None]], 'documents': [['動物界,脊索動物門,哺乳綱,靈長目,猴科,獼猴屬,台灣獼猴', '動物界,脊索動物門,哺乳綱,雙門齒目,袋鼠科,袋鼠屬,紅大袋鼠', '動物界,脊索動物門,哺乳綱,偶蹄目,牛科,鬣羚屬,台灣長鬃山羊', '動物界,脊索動物門,哺乳綱,肉食目,貓科,豹屬,花豹', '動物界,脊索動物門,哺乳綱,肉食目,貓科,石虎屬,石虎', '動物界,脊索動物門,哺乳綱,兔形目,兔科,穴兔屬,家兔']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}


In [25]:
# print(collection.peek(10)['documents'])
print(collection.peek(100)['ids'])
print(collection.peek(100)['documents'])

[]
[]


In [95]:
ollama.pull(model="llama3")
output = ollama.generate(
    model="llama3",
    prompt=f"以繁體中文回答\n使用這筆資料: {data}, 來對這個提問\n{prompt}\n產生對應的回應"
)
print(output['response'])

😊

根據提供的資料，以下是兩隻動物之間的相似度排序：

1. 紅大袋鼠 (Pseudocheirus peregrinus) - 90%
2. 台灣長鬃山羊 (Muntiacus reevesi) - 80%

這兩個動物都是哺乳綱（Mammalia）下的偶蹄目（Artiodactyla）的成員，並且都有袋鼠科（Macropodidae）或牛科（Bovidae）等的關係。

在這兩隻動物中，紅大袋鼠是最相近於白狗（Canis lupus familiaris）的，因為：

* 都是哺乳綱下的偶蹄目。
* 都有袋鼠科或牛科的關係。
* 都有四肢和尾巴。

因此，根據資料庫中的資料，我們可以認為紅大袋鼠與白狗之間的相似度最高。 👍


In [88]:
delete_collection("docs")